In [16]:
import tensorflow as tf
import pandas as pd
import keras
from sklearn.model_selection import train_test_split
from keras import *
from keras.src.layers import Dense, LeakyReLU, Dropout,GlobalAveragePooling2D
from keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import numpy as np
import zipfile
import os
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [17]:
data_dir = 'C:/Users/lisat/OneDrive/Documents/GitHub/quality-control/linsen_data/train'

image_size = (512,512)

train_data, val_data = keras.utils.image_dataset_from_directory(
    data_dir,
    labels="inferred",
    validation_split=0.2,
    subset="both",
    seed=6568,
    color_mode="rgb",
    batch_size=32,
    image_size=image_size,
    shuffle=True,
)

def normalize_data(data, label):
    data = tf.cast(data, tf.float32) / 255.0
    return data, label

def preprocess_data(data, label):
    data = preprocess_input(tf.cast(data, tf.float32))
    return data, label

data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal_and_vertical"),
    layers.RandomRotation(0.2),
    layers.RandomZoom(0.2),
    layers.RandomContrast(0.1),
])

# Apply data augmentation to training data
train_data = train_data.map(lambda x, y: (data_augmentation(x, training=True), y))

train_data = train_data.map(normalize_data)
val_data = val_data.map(normalize_data)

model = Sequential([
        # First Convolutional Block
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Second Convolutional Block
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Third Convolutional Block
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.BatchNormalization(),
    layers.MaxPooling2D((2, 2)),

    # Attention Layer (optional)
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.4),

    # Output Layer
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
                loss=tf.keras.losses.BinaryCrossentropy(),
                metrics=['accuracy'])
print(model.summary())


Found 1404 files belonging to 2 classes.
Using 1124 files for training.
Using 280 files for validation.


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_19 (Conv2D)              │ (None, 510, 510, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_19          │ (None, 510, 510, 32)   │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_19 (MaxPooling2D) │ (None, 255, 255, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_20 (Conv2D)              │ (None, 253, 253, 64)   │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_20          │ (None, 253, 253, 64)   │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_20 (MaxPooling2D) │ (None, 126, 126, 64)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_21 (Conv2D)              │ (None, 124, 124, 128)  │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_21          │ (None, 124, 124, 128)  │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_21 (MaxPooling2D) │ (None, 62, 62, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_6      │ (None, 128)            │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_12 (Dense)                │ (None, 128)            │        16,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_6 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_13 (Dense)                │ (None, 1)              │           129 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 110,785 (432.75 KB)

 Trainable params: 110,337 (431.00 KB)

 Non-trainable params: 448 (1.75 KB)

None


In [ ]:
# Change name before training!
epochs = 25

callbacks = [
    keras.callbacks.ModelCheckpoint(
        filepath="SR_BigBoi2_BW2_Dropout0.3_L-Data_512_E{epoch}.keras",
        save_weights_only=False    # Ensure both weights and model architecture are saved
    ), 
    keras.callbacks.EarlyStopping(
        monitor= 'val_accuracy',     # Monitor validation accuracy
        patience=4,                 # Stop after 4 epochs of no improvement
        restore_best_weights=True   # Restore the best weights
    )
]

history = model.fit(
    train_data,
    epochs=epochs,
    callbacks=callbacks,
    validation_data=val_data
)

current_description = '(Grayscale)'
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('CNN Model Loss ' + current_description)
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper right')
# plt.ylim(0, 0.6)
plt.show()

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('CNN Model Accuracy ' + current_description)
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='lower right')
plt.ylim(0, 1)
plt.show()

Epoch 1/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 224s 6s/step - accuracy: 0.7636 - loss: 0.4816 - val_accuracy: 0.3071 - val_loss: 0.7042
Epoch 2/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 223s 6s/step - accuracy: 0.8144 - loss: 0.4220 - val_accuracy: 0.5786 - val_loss: 0.6945
Epoch 3/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 231s 6s/step - accuracy: 0.8610 - loss: 0.3265 - val_accuracy: 0.5929 - val_loss: 0.6810
Epoch 4/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 216s 6s/step - accuracy: 0.8746 - loss: 0.2903 - val_accuracy: 0.3357 - val_loss: 0.8117
Epoch 5/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 233s 6s/step - accuracy: 0.8074 - loss: 0.3826 - val_accuracy: 0.5821 - val_loss: 0.7137
Epoch 6/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 261s 7s/step - accuracy: 0.8652 - loss: 0.2913 - val_accuracy: 0.5786 - val_loss: 0.7467
Epoch 7/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 302s 8s/step - accuracy: 0.8739 - loss: 0.2874 - val_accuracy: 0.6036 - val_loss: 0.6235
Epoch 8/25
36/36 ━━━━━━━━━━━━━━━━━━━━ 364s 10s/step - accuracy: 0.8523 - loss: 0.3306 - val_accuracy: 0.7036 - 

In [ ]:
test_path = 'C:/Users/lisat/OneDrive/Documents/GitHub/quality-control/linsen_data/test'

import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Function to preprocess a single image
def preprocess_image(image_path):
    img = load_img(image_path)  # Load the image
    img_array = img_to_array(img)  # Convert to numpy array
    img_array = img_array / 255.0  # Normalize pixel values to [0, 1] if required
    img_array = tf.expand_dims(img_array, axis=0)  # Add batch dimension
    return img_array

# Function to load images and labels from a folder
def load_images_and_labels(folder_path):
    images = []
    labels = []
    file_names = []
    for label, subfolder in enumerate(['good', 'not-good']):  # good = 0, not-good = 1
        subfolder_path = os.path.join(folder_path, subfolder)
        for filename in os.listdir(subfolder_path):
            if filename.endswith('.jpg') or filename.endswith('.png'):  # Valid image files
                image_path = os.path.join(subfolder_path, filename)
                img_array = preprocess_image(image_path)
                images.append(img_array)
                labels.append(label)
                file_names.append(filename)
    return np.vstack(images), np.array(labels), file_names


# Load images and labels
x_test, y_true, file_names = load_images_and_labels(test_path)

# Predict on test images
y_pred_prob = model.predict(x_test)  # Get probabilities
y_pred = (y_pred_prob > 0.5).astype(int).flatten()  # Convert probabilities to binary predictions

print("Predictions for each file:")
for file_name, prob, pred, true_label in zip(file_names, y_pred_prob.flatten(), y_pred, y_true):
    print(f"File: {file_name}, Probability: {prob:.4f}, Predicted: {pred}, True Label: {true_label}")

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

# Print results
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")